# Codequalität

Jupyter-Notebooks wurden ursprünglich entwickelt, um als *Labor* für Datenanalyse verwendet werden zu können. Häufig wirst du sie allerdings in deinen normalen Workflow einbauen und damit wie in einer web-basierten IDE arbeiten.

Dabei gibt es einige Besonderheiten zu beachten, die du hier gleich kennenlernen wirst.

## Out-of-order Execution

Normalerweise führst du die Zellen in einem Jupyter-Notebook von *oben nach unten aus*.

Häufig beginnst du mit einem `import`:

In [ ]:
import pandas as pd
import numpy as np

Hier erzeugst du jetzt einen `DataFrame`, der eine Sinusfunktion in der y-Koordinate enhält. Die x-Koordinate ist linear unterteilt von `-5` bis `5` in Schritten von `0.05`, damit sich eine stetige Kurve ergibt:

In [ ]:
df = pd.DataFrame({ "x": np.arange(-5, 5, 0.05), 
                    "y": np.sin(np.arange(-5, 5, 0.05)) })

Wenn du dir das Ergebnis als Diagramm ausgeben lässt, siehst du, dass das gut geklappt hat.

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
sns.lineplot(x="x", y="y", data=df)

Jetzt interessierst du dich für die *Exponentialfunktion* und lässt diese genau wie oben ausgeben:

In [ ]:
df = pd.DataFrame({ "x": np.arange(-5, 5, 0.05), 
                    "y": np.exp(np.arange(-5, 5, 0.05)) })
df.set_index("x").plot()

Der *Stil* gefällt dir noch nicht, der lässt sich leicht ändern:

In [ ]:
import matplotlib
matplotlib.style.use('ggplot')

Wie sieht das jetzt aus?

In [ ]:
df.set_index("x").plot()

Das ist viel besser, und du willst das nun auch auf den Sinus oben anwenden!

Dabei entstehen gleich mehrere Probleme:
* Der Sinus ist verschwunden und stattdessen wird auch die Exponentialfunktion angewendet.
* Beim nächsten Durchlauf ist der Style des Sinus-Plot wieder der ursprüngliche.

Das Notebook ist also *inkonsistent* geworden. Und das ist noch ein relativ einfacher Fall.

Nutze nun `seaborn` zum Plot:

In [ ]:
import seaborn as sns

Die Aufrufsyntax ist etwas anders, aber auch intuitiv:

In [ ]:
sns.lineplot(x="x", y="y", data=df)

Noch besser mit den Achsenbeschriftungen! Setze das auch oben ein.

Nun entsteht ein neues Problem, das Notebook ist in Zukunft so gar nicht mehr ablauffähig, weil du oben `sns` vor dem Import verwendet. 

Diese Fälle treten leider sehr häufig auf und sind auf die *Ablaufreihenfolge* zurückzuführen. Sie haben auch noch andere Gründe:
* Nutzung von globalen Variablen
* Nutzung von globalen Einstellungen
* Nutzung von globalem Import

Prinzipiell könntest du das natürlich alles vermeiden und den Code in *Funktionen kapseln*. Dadurch geht allerdings viel von der Interaktivität der Jupyter-Notebooks verloren und der Spaß bleibt auf der Strecke.

Um das Problem an der Wurzel zu bekämpfen, solltest du alle Notebooks nach "Fertigstellung" nochmal von vorne laufen lassen. Dazu gibt es die Funktion *Kernel / Restart & Run All*. Zugegebenermaßen ist es *nervig*, das immer wieder aufzurufen. Du solltest es dir trotzdem auf jeden Fall angewöhnen. Es ist sehr unerfreulich, wenn du ein Notebook nach vielen Monaten wieder benötigst und *erst dann* merkst, dass es nicht mehr funktioniert. Nun hast du es nämlich viel schwerer, den Fehler zu finden, weil du dich erst wieder mit dem Code vertraut machen musst.

## Unvollständige Datenmanipulation

`pandas` ermöglicht dir eine sehr komfortable Selektion (Filterung) von Teildatensets: 

In [ ]:
part = df[df["x"] > 0]
part

Das funktioniert, weil `df["x"] > 0` ein `boolean`-Array zurückliefert und dieses als Index verwendet nur die Werte zurückliefert, bei denen der Wert `True` ist

In [ ]:
df["x"] > 0

Nun kannst du in dem *Teildatenset* eine neue Variable setzen:

In [ ]:
part["z"] = np.log(part["x"])

Die Fehlermeldung *verheißt nichts Gutes*. Die Variable `part` ist zwar richtig gesetzt, aber auf `df` hatte das keinen Effekt:

In [ ]:
part

In [ ]:
df

Das Problem liegt daran, dass `df[df["x] > 0]` lediglich *Verweise* auf den `DataFrame` anlegt. Du kannst die Fehlermeldung vermeiden, indem du noch ein `.copy()` einfügst:

In [ ]:
part = df[df["x"] > 0].copy()
part["z"] = np.log(part["x"])
part

Achtung: `.copy()` benötigt zusätzlichen Speicherplatz. Oft wirst du es doch als eine enorm praktische Methode verwenden, um *nebenwirkungsfrei* auf Teilmengen von Daten zugreifen zu können. 

## `git`-Integration

Wenn du dich mit Softwareentwicklung beschäftigst, bist du sicher auch schon mit [`git`](https://git-scm.com/) vertraut. Ursprünglich für den Linux-Kernel entwickelt, ist es heute der *de-facto-Standard* für Konfigurationsmanagement und Versionskontrolle.

Leider eignen sich Jupyter-Notebook nicht besonders gut, um in `git` eingecheckt zu werden. Das liegt daran, dass neben den Code- und Dokumentationszellen auch alle Ausgaben von Python in den `.ipynb`-Files enthalten sind. Dadurch kannst du die echten Änderungen von einer auf die nächste Version nicht mehr erkennen.

Es gibt sog. *git hooks*, die diesen Code entfernen. Alternativ kannst du vor jedem Speichern des Notebooks die Funktion *Kernel / Restart & Clear Output* aufrufen. Eine weitere Möglichkeit ist die Verwendung eines spezielle `diff`-Programms, das die Änderungen in den Outputs nicht berücksichtigt.

## Unterschiedliche Kernel

Wenn du viel mit unterschiedlichen Projekten arbeitet, wirst du  mit unterschiedlichen Python-Paketen arbeiten wollen. Python selbst bietet mit dem `venv`-Paket dafür einen Mechanismus, der es dir erlaubt, mit unterschiedlichen Umgebungen parallel zu arbeiten. Wenn du dich daran gewöhnt hast, kannst du sehr komfortabel damit umgehen.

Glücklicherweise kannst du diesem Mechanismus auch in Jupyter-Notebooks nützen. Auch hier kannst du `venv` anlegen und die dem Notebook-Server bekannt machen. Dieser stellt dir dann unterschiedliche Kernel bereit, die mit ganz unterschiedlichen Paketen (und Paketversionen) ausgestattet sind.

Wie das genau funktioniert, ist sehr stark abhängig vom Betriebssystem und der verwendeten Python-installation (Betriebssystem oder Anaconda). Mithilfe von Suchmaschinen findest du (z.B. auf Stackoverflow) eine genau Anleitung, wie das in deinem konkreten Szenario funktionieren kann.